In [1]:
# hide
from nbdev.showdoc import *

# Easy submission example

In [2]:
import pandas as pd
from nbdev import show_doc
from numerai_blocks.key import Key, load_key_from_json
from numerai_blocks.submission import NumeraiClassicSubmittor, NumeraiSignalsSubmittor

In order to use a Submittor you should first create a `Key` object which handles credentials.
There are two ways to create a `Key`:

**1. Initialize `Key` with `pub_id` and `secret_key` from memory.**

In [3]:
# Random credentials
pub_id = "UFVCTElDX0lE"
secret_key = "U1VQRVJfU0VDUkVUX0tFWQ=="
key = Key(pub_id=pub_id, secret_key=secret_key)
key

🔑 Numerai Auth key initialized with pub_id = 'UFVCTElDX0lE' 🔑

Numerai Auth Key. pub_id = 'UFVCTElDX0lE'

**2. Load credentials from `.json` file with `load_key_from_json`.**

JSON file should have the following format:
```json
{"pub_id": "PUBLIC_ID", "secret_key": "SECRET_KEY"}
```
We recommend loading from `.json`. With this method you only have to save your credentials in one (safe) place and avoid leaving reference to a secret key in Python code.

In [4]:
# Random credentials
key = load_key_from_json("../nbs/test_assets/test_credentials.json")
key

🔑 Numerai Auth key initialized with pub_id = 'UFVCTElDX0lE' 🔑

Numerai Auth Key. pub_id = 'UFVCTElDX0lE'

In [5]:
# hide_input
show_doc(Key)

<h2 id="Key" class="doc_header"><code>class</code> <code>Key</code><a href="https://github.com/crowdcent/numerai_blocks/tree/main/numerai_blocks/key.py#L12" class="source_link" style="float:right">[source]</a></h2>

> <code>Key</code>(**`pub_id`**:`str`, **`secret_key`**:`str`)

Immutable Numerai credentials.

--------------------------------------------------

## 1. Numerai Classic

Submissions can be done in 2 lines of code. To initialize the submittor object, pass a directory path for saving submissions and a `Key` object.

`NumeraiClassicSubmittor.full_submission` will perform:
 1. Checks to prevent surprise behavior (including value range and column validity)
 2. Saving to CSV
 3. Uploading with `numerapi`.

The `dataf` argument can be either a `pd.DataFrame` or `NumerFrame`.

For multi-target, specify a list of targets in `cols`.

In [6]:
submittor = NumeraiClassicSubmittor(directory_path="sub_current_round", key=key)
# Your prediction file with 'id' as index and defined 'cols' below.
dataf = pd.DataFrame(columns=["prediction"])
# Only works with valid key credentials
# submittor.full_submission(dataf=dataf,
#                           cols="prediction",
#                           file_name="submission.csv",
#                           model_name="integration_test",
#                           version=2
#                           )

No existing directory found at 'sub_current_round'. Creating directory...

For this example, `submission.csv` can now be found on path `sub_current_round/submission.csv` and predictions are submitted for `integration_test`. Of course the uploading only works with valid credentials in the `Key`.


Alternatively, you can use `NumeraiClassicSubmittor` to save a csv and also benefit from all the checks performed by the Submittor.

In [7]:
submittor.save_csv(dataf=dataf, cols='prediction', file_name='submission.csv')

📄 Saving predictions CSV to 'sub_current_round/submission.csv'. 📄

In [8]:
#hide_input
show_doc(NumeraiClassicSubmittor)

<h2 id="NumeraiClassicSubmittor" class="doc_header"><code>class</code> <code>NumeraiClassicSubmittor</code><a href="https://github.com/crowdcent/numerai_blocks/tree/main/numerai_blocks/submission.py#L117" class="source_link" style="float:right">[source]</a></h2>

> <code>NumeraiClassicSubmittor</code>(**`directory_path`**:`str`, **`key`**:[`Key`](/numerai_blocks/key.html#Key), **\*`args`**, **\*\*`kwargs`**) :: [`BaseSubmittor`](/numerai_blocks/submission.html#BaseSubmittor)

Submit for Numerai Classic.
:param directory_path: Base directory to save and read prediction files from.
:param key: Key object (numerai-blocks.key.Key) containing valid credentials for Numerai Classic.
*args, **kwargs will be passed to NumerAPI initialization.

----------------------------------------------------

## 2. Numerai Signals

`NumeraiSignalsSubmittor` is very similar to `NumeraiClassicSubmittor`, but has a few additional checks specific to Signals. Mainly, it checks if the data contains a valid ticker column (`"cusip"`, `"sedol"`, `"ticker"`, `"numerai_ticker"` or `"bloomberg_ticker"`) and a `'signal'` column.

`NumeraiSignalsSubmittor.full_submission` handles checks, saving of CSV and uploading with `numerapi`.

In [9]:
submittor = NumeraiSignalsSubmittor(directory_path="sub_current_round", key=key)
# Your prediction file with 'id' as index, a valid ticker column and signal column below.
dataf = pd.DataFrame(columns=['bloomberg_ticker', 'signal'])
# Only works with valid key credentials
# submittor.full_submission(dataf=dataf,
#                           cols=["bloomberg_ticker", "signal"],
#                           file_name="submission.csv",
#                           model_name="integration_test"
#                           )

You also have the option to use `.save_csv` separately and still benefit from the checks the submittor performs. It will save the columns `'bloomberg_ticker'` and `'signal'` by default. If your use case is different define the columns you want to save in the `cols` argument.

In [10]:
submittor.save_csv(dataf=dataf, file_name='submission.csv')

📄 Saving Signals predictions CSV to 'sub_current_round/submission.csv'. 📄

In [11]:
#hide_input
show_doc(NumeraiSignalsSubmittor)

<h2 id="NumeraiSignalsSubmittor" class="doc_header"><code>class</code> <code>NumeraiSignalsSubmittor</code><a href="https://github.com/crowdcent/numerai_blocks/tree/main/numerai_blocks/submission.py#L152" class="source_link" style="float:right">[source]</a></h2>

> <code>NumeraiSignalsSubmittor</code>(**`directory_path`**:`str`, **`key`**:[`Key`](/numerai_blocks/key.html#Key), **\*`args`**, **\*\*`kwargs`**) :: [`BaseSubmittor`](/numerai_blocks/submission.html#BaseSubmittor)

Submit for Numerai Signals
:param directory_path: Base directory to save and read prediction files from.
:param key: Key object (numerai-blocks.key.Key) containing valid credentials for Numerai Signals.
*args, **kwargs will be passed to SignalsAPI initialization.

-----------------------------------------------------

When you are done with submissions and don't need the submission anymore you can remove the submission directory with 1 line. Convenient if you have automated jobs and want to avoid clutter due to saving submission files for every round.

In [12]:
# Clean up environment
submittor.remove_base_directory()

⚠ Deleting directory for 'NumeraiSignalsSubmittor' ⚠
Path: 
'/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/edu_nbs/sub_current_round'

In [13]:
#hide_input
show_doc(submittor.remove_base_directory)

<h4 id="BaseIO.remove_base_directory" class="doc_header"><code>BaseIO.remove_base_directory</code><a href="https://github.com/crowdcent/numerai_blocks/tree/main/numerai_blocks/download.py#L31" class="source_link" style="float:right">[source]</a></h4>

> <code>BaseIO.remove_base_directory</code>()

Remove directory with all contents.